In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from collections import defaultdict
from typing import List, Tuple
from scipy.stats import skew, kurtosis

In [2]:
# Load the data
data = {
    "eod_balance_training": pd.read_csv(
        "../berkabank/primary/eod_balance_training.csv"
    ),
    "incidents": pd.read_csv("../berkabank/primary/incidents.csv"),
}

In [3]:
from dataclasses import dataclass
import pandas as pd
from typing import Dict, List, Any, Callable


@dataclass
class IncidentFeatures:
    incidents: pd.DataFrame
    column_mapping: dict

    def run(self):
        print("----- Running IncidentFeatures...")
        incident_features = pd.DataFrame()
        print("----- IncidentFeatures completed.")
        return incident_features


@dataclass
class EODBFeatures:
    eod_balance_training: pd.DataFrame
    column_mapping: dict
    aggregations: dict

    def run(self):
        print("----- Running EODBFeatures...")
        eodb_features_output = pd.DataFrame()
        print("----- EODBFeatures completed.")
        return eodb_features_output


@dataclass
class PrimaryFeatures:
    incident_features: IncidentFeatures
    eodb_features: EODBFeatures

    def __post_init__(self):
        print("--- Initializing PrimaryFeatures...")
        self.incident_features_output = self.incident_features.run()
        self.eodb_features_output = self.eodb_features.run()
        print("--- PrimaryFeatures initialized.")

    def run(self):
        print("--- Running PrimaryFeatures...")
        primary_features_output = pd.DataFrame()
        print("--- PrimaryFeatures completed.")
        return primary_features_output


@dataclass
class DerivedFeatures:
    primary_features: PrimaryFeatures

    def __post_init__(self):
        print("--- Initializing DerivedFeatures...")
        self.primary_features_output = self.primary_features.run()
        print("--- DerivedFeatures initialized.")

    def run(self):
        print("--- Running DerivedFeatures...")
        derived_features = pd.DataFrame()
        print("--- DerivedFeatures completed.")
        return derived_features


@dataclass
class FeatureEngineering:
    primary_features: PrimaryFeatures
    derived_features: DerivedFeatures

    def run(self):
        print("Running FeatureEngineering...")
        self.derived_features.run()
        print("FeatureEngineering completed.")

In [4]:
column_mapping = {
    "incidents": {
        "account_id": "account_id",
        "incident_date": "incident_date",
        "district_id": "district_id",
        "t0": "t0",
        "t1": "t1",
    },
    "eod_balance_training": {
        "account_id": "account_id",
        "balance_date": "balance_date",
        "end_of_day_balance": "end_of_day_balance",
        "daily_amount_flow": "daily_amount_flow",
        "n_transactions": "n_transactions",
        "days_since_account_creation": "days_since_account_creation",
        "low_balance_streak": "low_balance_streak",
        "district_id": "district_id",
    },
}

In [5]:
df = (
    data["eod_balance_training"]
    .loc[
        :,
        [
            "account_id",
            "balance_date",
            "end_of_day_balance",
            "daily_amount_flow",
            "n_transactions",
        ],
    ]
    .copy()
)

In [6]:
df.head()

,account_id,balance_date,end_of_day_balance,daily_amount_flow,n_transactions
0,576,1993-01-11,7107.0,6207.0,1.0
1,576,1993-01-12,7107.0,0.0,1.0
2,576,1993-01-13,7107.0,0.0,1.0
3,576,1993-01-14,7107.0,0.0,1.0
4,576,1993-01-15,7107.0,0.0,1.0


In [7]:
df["flow_category"] = (
    df["daily_amount_flow"].gt(0).replace({True: "inflow", False: "outflow"})
)
df["daily_amount_inflow"] = df["daily_amount_flow"].clip(lower=0)
df["daily_amount_outflow"] = df["daily_amount_flow"].clip(upper=0).abs()

In [8]:
import pandas as pd
import re


def percentile(p):
    def percentile_(x):
        return np.nanpercentile(x, p)

    percentile_.__name__ = "percentile_%s" % p
    return percentile_


def kurtosis(series):
    return pd.Series(series).kurt()


def get_function_name(func):
    if callable(func):
        return func.__name__
    else:
        return func


def create_aggregations(
    max_t: int = 60,
    min_t: int = 5,
    step: int = 5,
    agg_funcs: List[str] = ["mean", "max", "sum", "std", "median"],
    features: List[str] = ["daily_amount_flow", "n_transactions"],
) -> List[str]:
    """Create aggregation names based on the given parameters.

    Args:
        max_t (int, optional): Maximum time period. Defaults to 60.
        min_t (int, optional): Minimum time period. Defaults to 5.
        agg_funcs (List[str], optional): Aggregation functions. Defaults to ["mean", "max", "sum", "std", "median"].
        category (List[str], optional): Categories. Defaults to ["inflow", "outflow"].
        features (List[str], optional): Features. Defaults to ["daily_amount_flow"].

    Returns:
        List[str]: Names of the aggregations.
    """
    return [
        f"f_{feature}__rolling_{get_function_name(func)}_{time_period}_days"
        for feature in features
        for time_period in range(min_t, max_t + 1, step)
        for func in agg_funcs
    ]


def parse_aggregation(aggregation):
    """Parse the name of an aggregation to get the feature, category, function, and time period.

    Args:
        aggregation (str): Name of the aggregation.

    Returns:
        tuple: feature, category, function, time period.
    """
    match = re.match(r"f_(\w+)__rolling_(\w+)_(\d+)_days", aggregation)
    feature, func, time_period = match.groups()
    time_period = int(time_period)

    return feature, func, time_period

In [38]:
aggregation_list = [
    "mean",
    "min",
    "max",
    "sum",
    "std",
    "median",
    "skew",
    "kurtosis",
] + [f"percentile_{p}" for p in range(10, 100, 25)]

aggregations = create_aggregations(
    max_t=90,
    min_t=10,
    step=10,
    agg_funcs=aggregation_list,
    features=["daily_amount_inflow", "daily_amount_outflow", "n_transactions"],
)

In [39]:
import warnings


def make_aggregations(
    df: pd.DataFrame,
    agg_funcs: List[str],
    aggregation_mapping: Dict[str, str] = None,
    index: str = "account_id",
) -> pd.DataFrame:
    """Create rolling aggregations for the given dataframe based on the name of the aggregation.

    Args:
        df (pd.DataFrame): DataFrame containing eos balance data.
        agg_funcs (List[str]): Feature names to create rolling aggregations for.

    Returns:
        pd.DataFrame: DataFrame with rolling aggregations for each period.
    """
    if aggregation_mapping is None:
        aggregation_mapping = {
            "mean": "mean",
            "min": "min",
            "max": "max",
            "sum": "sum",
            "std": "std",
            "median": "median",
            "skew": "skew",
            "kurtosis": "kurt",
            **{f"percentile_{p}": "quantile" for p in range(5, 100)},
        }

    for aggregation in agg_funcs:
        print(f"Applying aggregation function: {aggregation}")
        feature, func, time_period = parse_aggregation(aggregation)
        agg = aggregation_mapping.get(func)
        if agg is None:
            raise ValueError(f"Function {func} not found in the function mapping.")

        params = {}
        if func.startswith("percentile"):

            percentile = int(func.split("_")[1]) / 100
            params[agg] = percentile
        # Suppress the FutureWarning
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning)
            df[aggregation] = getattr(
                df.groupby(index)[feature].rolling(time_period), agg
            )(**params).reset_index(level=0, drop=True)

    return df

In [ ]:
df_agg = (
    make_aggregations(df, aggregations)
    .groupby("account_id")[aggregations]
    .agg(
        [
            "sum",
            "size",
            "mean",
            "std",
            "min",
            "max",
            "skew",
        ]
    )
)



In [41]:
df_agg.columns = [f"f_{col[0]}_{col[1]}" for col in df_agg.columns.values]

In [42]:
df_agg

,f_f_daily_amount_inflow__rolling_mean_10_days_sum,f_f_daily_amount_inflow__rolling_mean_10_days_size,f_f_daily_amount_inflow__rolling_mean_10_days_mean,f_f_daily_amount_inflow__rolling_mean_10_days_std,f_f_daily_amount_inflow__rolling_mean_10_days_min,f_f_daily_amount_inflow__rolling_mean_10_days_max,f_f_daily_amount_inflow__rolling_mean_10_days_skew,f_f_daily_amount_inflow__rolling_min_10_days_sum,f_f_daily_amount_inflow__rolling_min_10_days_size,f_f_daily_amount_inflow__rolling_min_10_days_mean,...,f_f_n_transactions__rolling_percentile_60_90_days_min,f_f_n_transactions__rolling_percentile_60_90_days_max,f_f_n_transactions__rolling_percentile_60_90_days_skew,f_f_n_transactions__rolling_percentile_85_90_days_sum,f_f_n_transactions__rolling_percentile_85_90_days_size,f_f_n_transactions__rolling_percentile_85_90_days_mean,f_f_n_transactions__rolling_percentile_85_90_days_std,f_f_n_transactions__rolling_percentile_85_90_days_min,f_f_n_transactions__rolling_percentile_85_90_days_max,f_f_n_transactions__rolling_percentile_85_90_days_skew
account_id,,,,,,,,,,,,,,,,,,,,,
1,48664.10,301,166.657877,264.940386,0.0,1261.92,2.427014,0.0,301,0.0,...,4.0,33.0,0.312050,4136.15,301,19.510142,9.077477,6.00,36.65,0.267806
2,276694.20,392,722.439164,1046.235074,0.0,3227.70,0.923714,0.0,392,0.0,...,70.4,135.0,-0.012400,32322.60,392,106.675248,18.728761,75.00,139.00,-0.019446
4,48762.50,275,183.317669,259.884361,0.0,555.30,0.739937,0.0,275,0.0,...,2.0,24.0,0.720329,2128.60,275,11.444086,7.867985,2.00,26.00,0.420745
6,77522.12,380,208.954501,305.457798,0.0,666.90,0.837856,0.0,380,0.0,...,36.0,91.0,-0.060047,20047.10,380,68.890378,16.822688,40.00,94.00,-0.163157
9,57065.30,376,155.491281,305.585765,0.0,1432.96,2.876551,0.0,376,0.0,...,4.0,45.0,-0.042682,7894.25,376,27.506098,12.303846,6.65,50.65,-0.061372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11320,217044.10,390,569.669554,865.327425,0.0,2612.70,0.994695,0.0,390,0.0,...,137.0,187.4,-0.022603,49846.50,390,165.602990,14.595454,140.00,189.65,-0.054699
11333,169770.18,398,436.427198,680.239456,0.0,2844.20,1.530857,0.0,398,0.0,...,57.4,111.0,0.014884,27144.45,398,87.846117,15.771725,61.00,114.00,-0.026391
11349,6318.20,14,1263.640000,2814.417923,0.0,6298.20,2.236015,0.0,14,0.0,...,NaN,NaN,NaN,0.00,14,NaN,NaN,NaN,NaN,NaN


In [43]:
def drop_missing_percentage(df, threshold=0.1):
    """
    Returns a list of column names from the DataFrame where less than 10% of the data is missing.

    Parameters:
    df (pandas.DataFrame): The DataFrame to process.

    Returns:
    list: A list of column names.
    """
    columns = (
        (df.isna().mean() < threshold)
        .replace({True: 1, False: np.nan})
        .dropna()
        .index.to_list()
    )
    return df.loc[:, columns]

In [56]:
@dataclass
class FeatureEliminationMissingRate:
    """Feature elimination class.

    Attributes:
        missing_rate_threshold (float): missing rate threshold

    Methods:
        run(X, y): fit the model

    Returns:
        list: reduced feature set
    """

    missing_rate_threshold: float

    def run(self, X: pd.DataFrame) -> pd.DataFrame:
        """Run the feature elimination process.

        Args:
            X (pd.DataFrame): input features
            y (np.array): target variable

        Returns:
            list: reduced feature set
        """
        missing_rate = X.isnull().mean()
        features_to_keep = missing_rate[
            missing_rate < self.missing_rate_threshold
        ].index
        return X[features_to_keep]




@dataclass
class FeatureEliminationKurtosis:
    """Feature elimination class.

    Attributes:
        kurtosis_threshold (float): kurtosis threshold

    Methods:
        run(X, y): fit the model

    Returns:
        list: reduced feature set
    """

    kurtosis_threshold: float

    def run(self, X: pd.DataFrame) -> pd.DataFrame:
        """Run the feature elimination process.

        Args:
            X (pd.DataFrame): input features

        Returns:
            list: reduced feature set
        """
        kurt = X.apply(kurtosis)
        features_to_keep = kurt[kurt < self.kurtosis_threshold].index
        return X[features_to_keep]
    


@dataclass
class FeatureEliminationCoV:
    """Feature elimination class.

    Attributes:
        cov_threshold (float): coefficient of variation threshold

    Methods:
        run(X, y): fit the model

    Returns:
        list: reduced feature set
    """

    cov_threshold: float

    def run(self, X: pd.DataFrame) -> pd.DataFrame:
        """Run the feature elimination process.

        Args:
            X (pd.DataFrame): input features
            y (np.array): target variable

        Returns:
            list: reduced feature set
        """
        cov = X.std() / X.mean()
        features_to_keep = cov[cov > self.cov_threshold].index
        return X[features_to_keep]


@dataclass
class FeatureEliminationPipeline:
    steps: Dict[str, Any]

    def run(self, X: pd.DataFrame, y: np.array) -> pd.DataFrame:
        """Run the feature elimination process.

        Args:
            X (pd.DataFrame): input features
            y (np.array): target variable

        Returns:
            list: reduced feature set
        """
        for step in self.steps:
            if y is None:
                X = self.steps[step].run(X)
            else:
                X = self.steps[step].run(X, y)
        return X


feature_elimination_pipeline = FeatureEliminationPipeline(
    {
        "missing_rate": FeatureEliminationMissingRate(0.1),
        "cov": FeatureEliminationCoV(0.8),
        "kurtosis": FeatureEliminationKurtosis(5),
    }
)

df_agg_fel = feature_elimination_pipeline.run(df_agg, y=None)

In [60]:
df_agg_fel.fillna(-1)

,f_f_daily_amount_inflow__rolling_mean_10_days_sum,f_f_daily_amount_inflow__rolling_mean_10_days_mean,f_f_daily_amount_inflow__rolling_mean_10_days_max,f_f_daily_amount_inflow__rolling_max_10_days_sum,f_f_daily_amount_inflow__rolling_max_10_days_mean,f_f_daily_amount_inflow__rolling_sum_10_days_sum,f_f_daily_amount_inflow__rolling_sum_10_days_mean,f_f_daily_amount_inflow__rolling_sum_10_days_max,f_f_daily_amount_inflow__rolling_std_10_days_sum,f_f_daily_amount_inflow__rolling_std_10_days_mean,...,f_f_n_transactions__rolling_percentile_35_90_days_mean,f_f_n_transactions__rolling_percentile_35_90_days_min,f_f_n_transactions__rolling_percentile_35_90_days_max,f_f_n_transactions__rolling_percentile_60_90_days_sum,f_f_n_transactions__rolling_percentile_60_90_days_mean,f_f_n_transactions__rolling_percentile_60_90_days_min,f_f_n_transactions__rolling_percentile_60_90_days_max,f_f_n_transactions__rolling_percentile_85_90_days_sum,f_f_n_transactions__rolling_percentile_85_90_days_mean,f_f_n_transactions__rolling_percentile_85_90_days_min
account_id,,,,,,,,,,,,,,,,,,,,,
1,48664.10,166.657877,1261.92,486425.4,1665.840411,486641.0,1666.578767,12619.2,1.538146e+05,526.762286,...,13.481368,2.0,28.15,3474.6,16.389623,4.0,33.0,4136.15,19.510142,6.00
2,276694.20,722.439164,3227.70,2742860.0,7161.514360,2766942.0,7224.391645,32277.0,8.687170e+05,2268.190557,...,97.271617,66.0,128.00,30891.0,101.950495,70.4,135.0,32322.60,106.675248,75.00
4,48762.50,183.317669,555.30,487625.0,1833.176692,487625.0,1833.176692,5553.0,1.542006e+05,579.701370,...,6.390860,1.0,19.00,1628.8,8.756989,2.0,24.0,2128.60,11.444086,2.00
6,77522.12,208.954501,666.90,775221.2,2089.545013,775221.2,2089.545013,6669.0,2.451465e+05,660.772152,...,60.561512,33.0,89.00,18835.0,64.725086,36.0,91.0,20047.10,68.890378,40.00
9,57065.30,155.491281,1432.96,562957.9,1533.945232,570653.0,1554.912807,14329.6,1.779889e+05,484.983417,...,21.187631,3.0,42.00,6965.4,24.269686,4.0,45.0,7894.25,27.506098,6.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11320,217044.10,569.669554,2612.70,2155701.0,5658.007874,2170441.0,5696.695538,26127.0,6.823041e+05,1790.824376,...,158.197674,133.0,183.00,48741.4,161.931561,137.0,187.4,49846.50,165.602990,140.00
11333,169770.18,436.427198,2844.20,1511809.7,3886.400257,1697701.8,4364.271979,28442.0,4.937509e+05,1269.282617,...,80.299838,56.0,106.15,25968.6,84.040777,57.4,111.0,27144.45,87.846117,61.00
11349,6318.20,1263.640000,6298.20,63182.0,12636.400000,63182.0,12636.400000,62982.0,1.997990e+04,3995.980542,...,-1.000000,-1.0,-1.00,0.0,-1.000000,-1.0,-1.0,0.00,-1.000000,-1.00


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis


# Create a larger dataset
np.random.seed(0)
n = 1000
data = {
    "account_id": np.random.choice(["A", "B", "C", "D", "E"], n),
    "daily_amount_flow": np.random.normal(100, 50, n),
    "n_transactions": np.random.poisson(10, n),
}
df = pd.DataFrame(data)

# Apply the groupby and aggregation
df_agg = df.groupby("account_id").agg(
    [
        "last",
        "sum",
        "size",
        "mean",
        "std",
        "min",
        "max",
        percentile(25),
        percentile(50),
        percentile(75),
        compute_skewness,
        compute_kurtosis,
    ]
)

df_agg